In [0]:
from pyspark.sql.functions import *
import requests
from datetime import datetime

In [0]:
def get_dollar_quotation(spark):
    """
    Função para obter a cotação atual do dólar usando a API do Banco Central
    """
    url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarDia(dataCotacao=@dataCotacao)"

    today = datetime.now().strftime('%m-%d-%Y')
    params = {'@dataCotacao': f"'{today}'", 'format': 'json'}
    
    try:

        response = requests.get(url, params=params)
        data = response.json()
        
        if 'value' in data and len(data['value']) > 0:
            quotation_data = [(
                datetime.now(),
                data['value'][0]['cotacaoCompra'],
                data['value'][0]['cotacaoVenda'],
                data['value'][0]['dataHoraCotacao']
            )]            
        
            df = spark.createDataFrame(
                quotation_data,
                ["timestamp", "buying_rate", "selling_rate", "quotation_datetime"]
            )            
        
            df = df.withColumn(
                "average_rate",
                (col("buying_rate") + col("selling_rate")) / 2
            )
            
            return df
        else:        
            default_data = [(datetime.now(), 0.0, 0.0, datetime.now())]
            return spark.createDataFrame(
                default_data,
                ["timestamp", "buying_rate", "selling_rate", "quotation_datetime"]
            )
            
    except Exception as e:
        print(f"Erro ao obter cotação: {str(e)}")
        return None

In [0]:
quotation_df = get_dollar_quotation(spark)

display(quotation_df)

# if quotation_df is not None:

#     df_with_brl = your_dataframe.withColumn(
#         "price_brl",
#         col("price_usd") * quotation_df.select("average_rate").first()[0]
#     )

# else:
#     print("Não foi possível obter a cotação do dólar")


timestamp,buying_rate,selling_rate,quotation_datetime,average_rate
2024-10-24T17:15:33.342371Z,5.7081,5.7087,2024-10-24 13:07:25.935,5.7084
